In [49]:
import os
from neo4j import GraphDatabase

In [50]:
URI = "bolt://localhost:7687"
AUTH = (os.environ.get("NEO4J_USERNAME"), os.environ.get("NEO4J_PASSWORD"))

In [55]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()
    print("Connection established.")

    with driver.session() as session:
        # Execute a Cypher query to create nodes
        session.run('MATCH (n) DETACH DELETE n')

        session.run('''
            CREATE (tnk1000:TANK {name: "Tank 1", status: "100% Full"}),
                    (mv3001:VALVE {name: "Valve 1", status: "Close"}),
                    (mv3002:VALVE {name: "Valve 2", status: "Open"}),
                    (tnk1000)-[:FEEDS_WATER_TO]->(mv3001),
                    (mv3001)-[:FEEDS_WATER_TO]->(mv3002)
        ''')
        print("Nodes created successfully.")

Connection established.
Nodes created successfully.


In [73]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

# Uncomment the below to use LangSmith. Not required.
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
# os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [74]:
os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "password"

In [75]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph()

# Import movie information

piping_query = """
            CREATE (tnk1000:TANK {name: "Tank 1", status: "100% Full"}),
                    (mv3001:VALVE {name: "Valve 1", status: "closed"}),
                    (mv3002:VALVE {name: "Valve 2", status: "open"}),
                    (tnk1000)-[:FEEDS_WATER_TO]->(mv3001),
                    (mv3001)-[:FEEDS_WATER_TO]->(mv3002)
"""

graph.query(piping_query)

[]

In [76]:
graph.refresh_schema()
print(graph.schema)

Node properties:
TANK {name: STRING, status: STRING}
VALVE {name: STRING, status: STRING}
Relationship properties:

The relationships:
(:TANK)-[:FEEDS_WATER_TO]->(:VALVE)
(:VALVE)-[:FEEDS_WATER_TO]->(:VALVE)


str

In [77]:
from langchain.chains import GraphCypherQAChain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0)
# llm = ChatOpenAI(model="o1-mini")
chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True, validate_cypher=True, allow_dangerous_requests=True)
response = chain.invoke({"query": "Why there is no water flow when 'Valve 2' is open?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (t:TANK)-[:FEEDS_WATER_TO]->(v1:VALVE {name: 'Valve 2'})-[:FEEDS_WATER_TO]->(v2:VALVE)
WHERE v1.status = 'open' AND (t.status <> 'active' OR v2.status <> 'open')
RETURN t.name AS Tank, t.status AS TankStatus, v2.name AS NextValve, v2.status AS NextValveStatus

Full Context:
[]

> Finished chain.


{'query': "Why there is no water flow when 'Valve 2' is open?",
 'result': "I don't know the answer."}

In [94]:
from crewai import Agent, Task, Crew
from crewai.tools import tool
from neo4j import GraphDatabase

# Assuming these are already set correctly for neo4j connection.
URI = "bolt://localhost:7687"  # Replace with your actual URI
AUTH = ("neo4j", "password")  # Replace with your actual authentication

@tool("Database_Access")
def database_access(cypher_code: str) -> str:
    """Useful for accessing neo4j graph database via cypher query and returning the result."""
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        driver.verify_connectivity()

        with driver.session() as session:
            result = session.run(cypher_code)  # Run the Cypher query
            records = result.data()  # Retrieve the result as a list of dictionaries

    # Convert the result to a string or return in a more suitable format
    return str(records)  # You can format or process the result as needed

# Define the schema directly in the code (no user input required)
schema = graph.schema

# Create the Cypher Generator Agent
cypher_generator = Agent(
    role="Cypher Generator",
    goal="An expert neo4j graph database cypher code generator based on input schema & user's {query}",
    backstory="""You are working for a water treatment plant and you are an expert advisor and troubleshooter in mechanical, electrical, control, and process systems.
    All of these are modeled into the neo4j graph database schema. You take in user {query} and translate it to Cypher code which will help answer the user's query.""",
    allow_delegation=False,  # No delegation to other agents
    verbose=True
)

# Create the Cypher Executer Agent
cypher_executer = Agent(
    role="Cypher Executer",
    goal="You obtain Cypher codes from Cypher generator and query it to neo4j",
    backstory="""You are an expert Cypher executer who takes in Cypher queries and executes them with the neo4j tool.""",
    allow_delegation=False,  # No delegation to other agents
    verbose=True
)

# Create the Cypher Result Interpreter Agent
cypher_result_interpreter = Agent(
    role="Cypher Result Interpreter",
    goal="You obtain the final result of the Cypher query to answer the user's {query}",
    backstory="""You are an expert Cypher result interpreter who translates the Cypher result to answer the user's {query}.""",
    allow_delegation=False,  # No delegation to other agents
    verbose=True
)

# Define the Tasks (no need for inputs/outputs in task definition)
task_cypher_generator = Task(
    description="Generate Cypher query to address user query using schema",
    role="Cypher Generator",
    agent=cypher_generator  # Ensure cypher_generator is defined correctly
)

task_cypher_executer = Task(
    description="Execute the Cypher query on the Neo4j database using the Database_Access tool",
    role="Cypher Executer",
    agent=cypher_executer,
    tools=[database_access]  # Explicitly pass the tool instance here
)

task_cypher_result_interpreter = Task(
    description="Interpret the result of the Cypher query to answer the user's question",
    role="Cypher Result Interpreter",
    agent=cypher_result_interpreter
)

# Create the Crew (the group of agents working together to complete the overall task)
crew = Crew(agents=[cypher_generator, cypher_executer, cypher_result_interpreter])

# Initialize the Crew and use crew.kickoff to execute tasks in sequence
crew.kickoff(inputs={'query': "Why is there no water flow when 'Valve 2' is open?"})


ValidationError: 1 validation error for Task
expected_output
  Field required [type=missing, input_value={'description': 'Generate...swer the user's query.)}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing